In [7]:
from SuperCNN import CNN

from SuperCNN import Parametros

from Auxiliares import takeinputs


In [8]:
val, pix, qcmval, qcmpix, pixelsconv, qcmpixconv = takeinputs()

convlay = [(1, "input"), (32, "relu")]

lay = [(64, "sigmoid"), (10, "softmax")]

parametros = Parametros(pix=pix, vales=val, qcmpix=qcmpix, qcmval=qcmval, infolay=lay, infoconvlay=convlay, padding=0, convrapide=True)

In [9]:
g = CNN(parametros)

g.train()


Percentage: 0.0



KeyboardInterrupt



In [ ]:
print(g.tauxlent())

In [ ]:
import pickle

with open('parameters.pickle', 'wb') as file:
    pickle.dump(g.parameters, file)
